In [74]:
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import csv
import re
import locale
pd.options.mode.chained_assignment = None  # default='warn'

In [75]:
def has_keyword(title, words):
    for kw in words:
        if kw in title:
            return True
    return False
def has_food_keyword(title):
    return has_keyword(title, ['餐饮', '百胜', '超市', '食品'])
def has_traffic_keyword(title):
    return has_keyword(title, ['地铁', '顺丰', '铁路'])
def has_fushi_keyword(title):
    return has_keyword(title, ['服饰', '曾宝琴'])

def predict_category(row):
    title = row['交易摘要']
    amt = row['amount']
    category = ''
    if float(amt) > 0:
        return category
        
    if has_food_keyword(title):
        category = '餐饮&外出就餐 > 餐饮/外出就餐'
    if has_traffic_keyword(title):
        category = '交通'
    if '燃气' in title:
        category = '账单 > 燃气'
    if has_fushi_keyword(title):
        category = '衣服 > 服装'
    if '发型' in title:
        category = '休闲 > 个人护理'
    if '淘票票' in title:
        category = '休闲 > 娱乐'
    if '中国移动' in title:
        category = '账单 > 电话'

    return category

In [76]:
def predict_payee(row):
    payee = row['交易摘要']
    segment_list = payee.split('-')
    if len(segment_list) > 1:
        payee = segment_list[1]
    segment_list = payee.split(' ')
    payee = segment_list[0]
    payee = re.sub('（.*）', '', payee)
    payee = re.sub('\(.*\)', '', payee)
    return payee

In [77]:
def predict_date(row):
    amt = row['amount']
    date_str = '2022/' + row['记账日']
    if float(amt) > 0:
        return date_str
    d = datetime.strptime(date_str, '%Y/%m/%d')
    d = d - timedelta(1)
    return d.strftime('%Y/%m/%d')

In [78]:
work_dir = 'C:/Users/wayne/Desktop/credit/'
filname = 'CreditCardReckoning2022-11'
credit_book = pd.read_csv(work_dir + filname + '.csv')
credit_book = credit_book[credit_book['交易日'].notnull()]
credit_book = credit_book[credit_book['交易日'] != 'SOLD']
credit_book.dropna(how='all', inplace=True)

locale.setlocale(locale.LC_ALL, '')
credit_book['amount'] = credit_book.apply(lambda row: locale.atof(row['人民币金额']) * -1, axis=1)
credit_book['category'] = credit_book.apply(lambda row: predict_category(row), axis=1)
credit_book['payee'] = credit_book.apply(lambda row: predict_payee(row), axis=1)
credit_book['date'] = credit_book.apply(lambda row: predict_date(row), axis=1)
credit_book.to_csv(work_dir + filname + '-out.csv', index=False)

credit_book['amount'].sum()

-3972.84